# Relation Global Mode Walkthrough

This notebook explains how `RelationSystem.solve(global_mode=True)` works and when it matters.

You will see:
- direct output evaluation when all inputs are known
- solving for one missing input
- coupled systems that require global mode
- nonlinear coupled systems that require `nsolve` (global mode only)

Assumptions:
- `Relation.expr` is a zero-residual expression (expression == 0).
- `Relation.variables[0]` is the preferred output variable.


## Global mode at a glance

In `RelationSystem.solve`, global mode changes three things:
- allows larger coupled components (up to 6 unknowns / 8 relations)
- enables numeric solving via `sympy.nsolve` when symbolic solving fails
- adds a per-variable coupled sweep (`solve_variable_systems`) when the main pass stalls


In [1]:
from __future__ import annotations

from pathlib import Path
import sys
import sympy as sp

repo_root = Path().resolve()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from fusdb.relation_class import Relation, RelationSystem
from fusdb.relation_util import symbol


## Case 1: Only the output is missing (direct evaluation)

If all inputs are numeric, the solver computes the output directly.


In [2]:
x = symbol("x")
y = symbol("y")

rel = Relation("y_from_x", ("y", "x"), y - (2 * x + 1))
system = RelationSystem([rel])
system.set("x", 3.0)

print(system.solve(global_mode=False))


{'y': 7.00000000000000, 'x': 3.00000000000000}


## Case 2: One missing input with known output

If the output is known and exactly one input is unknown, the solver solves for that input.


In [3]:
x = symbol("x")
y = symbol("y")

rel = Relation("solve_x", ("y", "x"), y - 2 * x)
system = RelationSystem([rel])
system.set("y", 10.0)

print(system.solve(global_mode=False))


{'y': 10.0000000000000, 'x': 5.00000000000000}


## Case 3: Coupled system needs global mode

Here all variables are unknown and no single relation can fire. Local mode only
tries small 2-unknown blocks, so it stalls. Global mode allows a larger coupled solve.


In [4]:
x = symbol("x")
y = symbol("y")
z = symbol("z")

relations = [
    Relation("eq1", ("x", "y", "z"), x + y + z - 9),
    Relation("eq2", ("x", "y", "z"), 2 * x - y + z - 8),
    Relation("eq3", ("x", "y", "z"), x + 2 * y - z - 3),
]

system = RelationSystem(relations)
print("local", system.solve(global_mode=False))

system = RelationSystem(relations)
print("global", system.solve(global_mode=True))


local {'x': x, 'y': y, 'z': z}
global {'x': 3.00000000000000, 'y': 2.00000000000000, 'z': 4.00000000000000}


## Case 4: Nonlinear coupled system uses nsolve (global mode)

Global mode allows numeric solving when symbolic solving fails. Provide
`initial_guesses` so `nsolve` can converge.


In [5]:
x = symbol("x")
y = symbol("y")

relations = [
    Relation("eq1", ("x", "y"), sp.sin(x) - y, initial_guesses={"x": 0.5, "y": 0.5}),
    Relation("eq2", ("x", "y"), x + y - 1, initial_guesses={"x": 0.5, "y": 0.5}),
]

system = RelationSystem(relations)
print("local", system.solve(global_mode=False))

system = RelationSystem(relations)
print("global", system.solve(global_mode=True))


local {'x': x, 'y': y}
global {'x': 0.510973429357289, 'y': 0.489026570642711}


## Case 5: tau_E / P_loss coupled power law

This mirrors the pattern `tau_E = C * P_loss**alpha_p` and `P_loss = W_th / tau_E`.
Both `tau_E` and `P_loss` are unknown; `W_th` is known.


In [6]:
P_loss = symbol("P_loss")
tau_E = symbol("tau_E")
W_th = symbol("W_th")
x = symbol("x")

alpha_p = 0.7
C = 0.9

relations = [
    Relation("tau_from_ploss", ("tau_E", "P_loss"), tau_E - C * P_loss**alpha_p),
    Relation("ploss_from_tau", ("P_loss", "tau_E", "W_th"), P_loss - W_th / tau_E),
]

system = RelationSystem(relations)
system.set("W_th", 100.0)
print("local", system.solve(global_mode=False))

system = RelationSystem(relations)
system.set("W_th", 100.0)
print("global", system.solve(global_mode=True))


local {'tau_E': tau_E, 'P_loss': P_loss, 'W_th': 100.000000000000}
global {'tau_E': 6.26056084691640, 'P_loss': 15.9730098379861, 'W_th': 100.000000000000}


In [7]:
P_loss = symbol("P_loss")
tau_E = symbol("tau_E")
W_th = symbol("W_th")
x = symbol("x")

alpha_p = 0.7
C = 0.9

relations = [
    Relation("tau_from_ploss", ("tau_E", "P_loss"), tau_E - C * P_loss**alpha_p),
    Relation("ploss_from_tau", ("P_loss", "tau_E", "W_th"), P_loss - W_th / tau_E),
    Relation("additional_relation", ("P_loss", "x"), P_loss - 2 * x + 10),
]

system = RelationSystem(relations)
system.set("W_th", 100.0)
print("local", system.solve(global_mode=False))

system = RelationSystem(relations)
system.set("W_th", 100.0)
print("global", system.solve(global_mode=True))


local {'tau_E': tau_E, 'P_loss': P_loss, 'W_th': 100.000000000000, 'x': x}
global {'tau_E': 6.26056084691640, 'P_loss': 15.9730098379861, 'W_th': 100.000000000000, 'x': 12.9865049189930}


## Summary

- Direct evaluation and single-missing-input solving work in both modes.
- Global mode is for coupled systems that exceed local limits and for nonlinear systems that need `nsolve`.
- If `nsolve` is needed, supply `initial_guesses` per relation to guide convergence.
